Lenguaje de Manipulación de Datos
===

In [1]:
%load_ext bigdata
%hive_init

Hive initialized!


In [2]:
%%hive
DROP DATABASE IF EXISTS DMLdb CASCADE;
CREATE DATABASE DMLdb;
USE DMLdb;

CREATE TABLE persons (
    id INT,
    firstname VARCHAR(10),
    surname VARCHAR(10),
    birthday TIMESTAMP,
    color VARCHAR(9),
    quantity INT
);

---

## INSERT

    INSERT INTO TABLE tablename VALUES values_row [, values_row ...]
    
    values_row: 
       (value [, value ...])
       
Note que a diferencia de SQL, aca no es posible indicar para que columnas se van a insertar los valores, de tal manera que siempre se deben dar valores para todas las columnas.       

In [3]:
%%hive
--
-- Inserta el registro en la tabla.
-- Los valores están en el mismo orden de los campos.
--
INSERT INTO persons VALUES
   (1,"Vivian","Hamilton","1971-07-08","green",1);
    
SELECT * FROM persons;

1	Vivian	Hamilton	1971-07-08 00:00:00	green	1


In [4]:
%%hive
--
-- Inserta varios registros a la vez.
-- Los valores deben estar en el mismo orden de los campos.
--
INSERT INTO persons VALUES
    (2,"Karen","Holcomb","1974-05-23","green",4),
    (3,"Cody","Garrett","1973-04-22","orange",1);
    
SELECT * FROM persons;    

1	Vivian	Hamilton	1971-07-08 00:00:00	green	1
2	Karen	Holcomb	1974-05-23 00:00:00	green	4
3	Cody	Garrett	1973-04-22 00:00:00	orange	1


## UPDATE

    UPDATE tablename SET column = value [, column = value ...] [WHERE expression]

Véase https://cwiki.apache.org/confluence/display/Hive/Hive+Transactions

## DELETE

    DELETE FROM tablename [WHERE expression]
    
Véase https://cwiki.apache.org/confluence/display/Hive/Hive+Transactions    

## MERGE

    MERGE INTO <target table> AS T USING <source expression/table> AS S
    ON <boolean expression1>
    WHEN MATCHED [AND <boolean expression2>] THEN UPDATE SET <set clause list>
    WHEN MATCHED [AND <boolean expression3>] THEN DELETE
    WHEN NOT MATCHED [AND <boolean expression4>] THEN INSERT VALUES<value list>
    
Véase https://community.hortonworks.com/articles/97113/hive-acid-merge-by-example.html    

## ESCRITURA DE DATOS AL DISCO

    INSERT OVERWRITE [LOCAL] DIRECTORY directory1
      [ROW FORMAT row_format] [STORED AS file_format] 
      SELECT ... FROM ...

Si el archivo requerido ya tiene el formato deseado, es posible copiarlo directamente del sistema de archivos. En caso contrario, puede usar un INSERT para realizar la conversión de formato.

In [5]:
## crea el directorio persons-data
!rm -rf DMLexport-dir
!mkdir DMLexport-dir

In [6]:
%%hive
INSERT OVERWRITE LOCAL DIRECTORY 'DMLexport-dir'
ROW FORMAT 
DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM persons;

In [7]:
!ls DMLexport-dir/* 

DMLexport-dir/000000_0


In [8]:
!cat DMLexport-dir/* > DMLexport-data.csv
!cat DMLexport-data.csv

1,Vivian,Hamilton,1971-07-08 00:00:00,green,1
2,Karen,Holcomb,1974-05-23 00:00:00,green,4
3,Cody,Garrett,1973-04-22 00:00:00,orange,1


In [9]:
## se borra el archivo si existe
!rm -rf DMLexport-dir  DMLexport-data.csv

## LOAD DATA

Se crea el archivo CSV

In [10]:
%%writefile persons.csv
4,Roth,Fry,1975-01-29 00:00:00,black,1
5,Zoe,Conway,1974-07-03 00:00:00,blue,2
6,Gretchen,Kinney,1974-10-18 00:00:00,viole,1
7,Driscoll,Klein,1970-10-05 00:00:00,blue,5
8,Karyn,Diaz,1969-02-24 00:00:00,red,1
9,Merritt,Guy,1974-10-17 00:00:00,indigo,4
10,Kylan,Sexton,1975-02-29 00:00:00,black,4
11,Jordan,Estes,1969-12-07 00:00:00,indigo,4
12,Hope,Coffey,1973-12-24 00:00:00,green,5
13,Vivian,Crane,1970-08-27 00:00:00,gray,5
14,Clio,Noel,1972-12-12 00:00:00,red,5
15,Hope,Silva,1970-07-01 00:00:00,blue,5
16,Ayanna,Jarvis,1974-02-11 00:00:00,orange,5
17,Chanda,Boyer,1973-04-01 00:00:00,green,4
18,Chadwick,Knight,1973-04-29 00:00:00,yellow,1

Writing persons.csv


In [11]:
## prepeara el archivo CSV
## para cargarse en Hive
## reemplazando las comas por Ctr-A 
lines = open('persons.csv', 'r').read()
lines = lines.replace(',', chr(1))
open('persons', 'w').write(lines)

643

In [12]:
%%hive
--
-- el separador de campos debe ser ctrl-A
-- en Python es `chr(1)`
--
LOAD DATA LOCAL INPATH 'persons' INTO TABLE persons; 

In [13]:
%%hive
SELECT * FROM persons;

1	Vivian	Hamilton	1971-07-08 00:00:00	green	1
2	Karen	Holcomb	1974-05-23 00:00:00	green	4
3	Cody	Garrett	1973-04-22 00:00:00	orange	1
4	Roth	Fry	1975-01-29 00:00:00	black	1
5	Zoe	Conway	1974-07-03 00:00:00	blue	2
6	Gretchen	Kinney	1974-10-18 00:00:00	viole	1
7	Driscoll	Klein	1970-10-05 00:00:00	blue	5
8	Karyn	Diaz	1969-02-24 00:00:00	red	1
9	Merritt	Guy	1974-10-17 00:00:00	indigo	4
10	Kylan	Sexton	1975-03-01 00:00:00	black	4
11	Jordan	Estes	1969-12-07 00:00:00	indigo	4
12	Hope	Coffey	1973-12-24 00:00:00	green	5
13	Vivian	Crane	1970-08-27 00:00:00	gray	5
14	Clio	Noel	1972-12-12 00:00:00	red	5
15	Hope	Silva	1970-07-01 00:00:00	blue	5
16	Ayanna	Jarvis	1974-02-11 00:00:00	orange	5
17	Chanda	Boyer	1973-04-01 00:00:00	green	4
18	Chadwick	Knight	1973-04-29 00:00:00	yellow	1


## LOAD DATA con datos complejos

Apache Hive usa cuatro tipos de datos complejos. A coninuación se presentan los separadores por defecto.

* Record: \n o delimitador de filas
* FIELD (campos): Ctr+A o \001, delimita campos.
* ARRAY y STRUCT: Ctrl+B o \002, delimita elementos 
* MAP: Ctrl+C o \003, delimita parejas (key, value). 
* UNIONTYPE

In [14]:
%%hive
DROP TABLE IF EXISTS complextypes;
CREATE TABLE complextypes (
    id INT,
    arrayField ARRAY<STRING>, 
    structField STRUCT<c21:INT, c22:INT>,
    mapField MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

In [15]:
%%writefile complextypes.csv
1,A:B:C,1:2,key1#1:key2#2
2,E:F,3:4,key1#3:key3#4
3,G:H:I,5:6,key2#5:key3#6

Writing complextypes.csv


In [16]:
%%hive
LOAD DATA LOCAL INPATH 'complextypes.csv' INTO TABLE complextypes;
SELECT * FROM complextypes;

1	["A","B","C"]	{"c21":1,"c22":2}	{"key1":1,"key2":2}
2	["E","F"]	{"c21":3,"c22":4}	{"key1":3,"key3":4}
3	["G","H","I"]	{"c21":5,"c22":6}	{"key2":5,"key3":6}


In [17]:
%%hive
--
-- Selecciona el campo ARRAY
--
SELECT arrayField FROM complextypes;

["A","B","C"]
["E","F"]
["G","H","I"]


In [18]:
%%hive
SELECT arrayField[0] FROM complextypes;

A
E
G


In [19]:
%%hive
SELECT arrayField[2] FROM complextypes;

C
NULL
I


In [20]:
%%hive
SELECT structField FROM complextypes;

{"c21":1,"c22":2}
{"c21":3,"c22":4}
{"c21":5,"c22":6}


In [21]:
%%hive
SELECT structField.c21, structField.c22 FROM complextypes;

1	2
3	4
5	6


In [22]:
%%hive
SELECT mapField FROM complextypes;

{"key1":1,"key2":2}
{"key1":3,"key3":4}
{"key2":5,"key3":6}


In [23]:
%%hive
SELECT mapField['key1'] FROM complextypes;

1
3
NULL


In [24]:
%%hive
SELECT mapField['key3'] FROM complextypes;

NULL
4
6


In [25]:
!rm complextypes.csv

## INSERT ... SELECT ...

También es posible insertar datos desde queries. La sintaxis sería la siguiente:

    INSERT OVERWRITE TABLE bancos1
    SELECT * FROM Bancos
    WHERE .... ;
    

La clausula anterior es posible reescribirla como:

    FROM Bancos se 
    INSERT OVERWRITE TABLE bancos1
       SELECT * WHERE ....
    INSERT OVERWRITE TABLE bancos2
       SELECT * WHERE ... ;
       
generando múltiples INSERT a tablas diferentes.

In [26]:
%%hive
CREATE TABLE IF NOT EXISTS persons1 LIKE persons;

INSERT OVERWRITE TABLE persons1
SELECT * FROM persons
WHERE color='green';

SELECT * FROM persons1;

1	Vivian	Hamilton	1971-07-08 00:00:00	green	1
2	Karen	Holcomb	1974-05-23 00:00:00	green	4
12	Hope	Coffey	1973-12-24 00:00:00	green	5
17	Chanda	Boyer	1973-04-01 00:00:00	green	4


In [27]:
%%hive
CREATE TABLE IF NOT EXISTS persons2 LIKE persons;

FROM persons
INSERT OVERWRITE TABLE persons1
   SELECT * WHERE color='red'
INSERT OVERWRITE TABLE persons2
   SELECT * WHERE color='green';

SELECT * FROM persons1;

SELECT * FROM persons2;

8	Karyn	Diaz	1969-02-24 00:00:00	red	1
14	Clio	Noel	1972-12-12 00:00:00	red	5
1	Vivian	Hamilton	1971-07-08 00:00:00	green	1
2	Karen	Holcomb	1974-05-23 00:00:00	green	4
12	Hope	Coffey	1973-12-24 00:00:00	green	5
17	Chanda	Boyer	1973-04-01 00:00:00	green	4


## PARTITIONED BY

In [28]:
%%hive
DROP TABLE IF EXISTS persons3;

CREATE TABLE persons3 (
    id INT,
    firstname VARCHAR(10),
    surname VARCHAR(10),
    birthday TIMESTAMP,
    color VARCHAR(9)
)
PARTITIONED BY (quantity INT);

LOAD DATA LOCAL INPATH 'persons' INTO TABLE persons3; 

SELECT * FROM persons3;

4	Roth	Fry	1975-01-29 00:00:00	black	1
6	Gretchen	Kinney	1974-10-18 00:00:00	viole	1
8	Karyn	Diaz	1969-02-24 00:00:00	red	1
18	Chadwick	Knight	1973-04-29 00:00:00	yellow	1
5	Zoe	Conway	1974-07-03 00:00:00	blue	2
9	Merritt	Guy	1974-10-17 00:00:00	indigo	4
10	Kylan	Sexton	1975-03-01 00:00:00	black	4
11	Jordan	Estes	1969-12-07 00:00:00	indigo	4
17	Chanda	Boyer	1973-04-01 00:00:00	green	4
7	Driscoll	Klein	1970-10-05 00:00:00	blue	5
12	Hope	Coffey	1973-12-24 00:00:00	green	5
13	Vivian	Crane	1970-08-27 00:00:00	gray	5
14	Clio	Noel	1972-12-12 00:00:00	red	5
15	Hope	Silva	1970-07-01 00:00:00	blue	5
16	Ayanna	Jarvis	1974-02-11 00:00:00	orange	5


In [29]:
!ls -l /Volumes/Data/hive-warehouse/dmldb.db/persons3

total 0
drwxr-xr-x  4 jdvelasq  admin  128 Oct 20 13:39 quantity=1
drwxr-xr-x  4 jdvelasq  admin  128 Oct 20 13:39 quantity=2
drwxr-xr-x  4 jdvelasq  admin  128 Oct 20 13:39 quantity=4
drwxr-xr-x  4 jdvelasq  admin  128 Oct 20 13:39 quantity=5


In [30]:
!ls -l /Volumes/Data/hive-warehouse/dmldb.db/persons3/quantity=1

total 8
-rw-r--r--  1 jdvelasq  admin  164 Oct 20 13:39 000000_0


In [31]:
!cat /Volumes/Data/hive-warehouse/dmldb.db/persons3/quantity=1/000000_0

4RothFry1975-01-29 00:00:00black
6GretchenKinney1974-10-18 00:00:00viole
8KarynDiaz1969-02-24 00:00:00red
18ChadwickKnight1973-04-29 00:00:00yellow


In [32]:
!cat /Volumes/Data/hive-warehouse/dmldb.db/persons3/quantity=2/000000_0

5ZoeConway1974-07-03 00:00:00blue


In [33]:
!cat /Volumes/Data/hive-warehouse/dmldb.db/persons3/quantity=4/000000_0

9MerrittGuy1974-10-17 00:00:00indigo
10KylanSexton1975-03-01 00:00:00black
11JordanEstes1969-12-07 00:00:00indigo
17ChandaBoyer1973-04-01 00:00:00green


In [34]:
!cat /Volumes/Data/hive-warehouse/dmldb.db/persons3/quantity=5/000000_0

7DriscollKlein1970-10-05 00:00:00blue
12HopeCoffey1973-12-24 00:00:00green
13VivianCrane1970-08-27 00:00:00gray
14ClioNoel1972-12-12 00:00:00red
15HopeSilva1970-07-01 00:00:00blue
16AyannaJarvis1974-02-11 00:00:00orange


---

In [35]:
%%hive
-- limpia la base de datos
DROP DATABASE IF EXISTS DMLdb CASCADE;

In [36]:
!rm persons*

---